In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

import hues
from prettytable import PrettyTable

In [2]:
class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)

In [3]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1, 1)),
                   nn.Flatten())

net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

In [4]:
X = torch.rand(size=(1, 1, 96, 96))

tb = PrettyTable(['Index', 'Layer', 'Input Shape', 'Output Shape'])
for i, layer in enumerate(net):
    if isinstance(layer, nn.Sequential):
        for j, in_layer in enumerate(layer):
            input_shape = X.shape
            X = in_layer(X)
            output_shape = X.shape
            tb.add_row([f'{i}-{j}', in_layer.__class__.__name__, list(input_shape), list(output_shape)])
    else:
        input_shape = X.shape
        X = layer(X)
        output_shape = X.shape
        tb.add_row([f'{i}', layer.__class__.__name__, list(input_shape), list(output_shape)])

print(tb)

+-------+-------------------+------------------+------------------+
| Index |       Layer       |   Input Shape    |   Output Shape   |
+-------+-------------------+------------------+------------------+
|  0-0  |       Conv2d      |  [1, 1, 96, 96]  | [1, 64, 48, 48]  |
|  0-1  |        ReLU       | [1, 64, 48, 48]  | [1, 64, 48, 48]  |
|  0-2  |     MaxPool2d     | [1, 64, 48, 48]  | [1, 64, 24, 24]  |
|  1-0  |       Conv2d      | [1, 64, 24, 24]  | [1, 64, 24, 24]  |
|  1-1  |        ReLU       | [1, 64, 24, 24]  | [1, 64, 24, 24]  |
|  1-2  |       Conv2d      | [1, 64, 24, 24]  | [1, 192, 24, 24] |
|  1-3  |        ReLU       | [1, 192, 24, 24] | [1, 192, 24, 24] |
|  1-4  |     MaxPool2d     | [1, 192, 24, 24] | [1, 192, 12, 12] |
|  2-0  |     Inception     | [1, 192, 12, 12] | [1, 256, 12, 12] |
|  2-1  |     Inception     | [1, 256, 12, 12] | [1, 480, 12, 12] |
|  2-2  |     MaxPool2d     | [1, 480, 12, 12] |  [1, 480, 6, 6]  |
|  3-0  |     Inception     |  [1, 480, 6, 6]  |